In [2]:
from hasp.feature_pipeline import make_oversampled_feature_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from hasp.util import combine_classes
from hasp.augmenting_oversampler import AugmentingRandomOversampler
import pandas as pd
import numpy as np

## Data

In [6]:
from us8kdata.loader import UrbanSound8K
data = UrbanSound8K('../hasp/data')

In [7]:
train_fold = [samples for samples in data.fold_audio_generator(fold=[1, 2, 3, 6, 7, 8, 9, 10])]
val_fold = [samples for samples in data.fold_audio_generator(fold=[4])]
test_fold = [samples for samples in data.fold_audio_generator(fold=[5])]

In [8]:
train_y = data.filter_metadata(fold=[1, 2, 3, 6, 7, 8, 9, 10]).classID
val_y = data.filter_metadata(fold=[4]).classID
test_y = data.filter_metadata(fold=[5]).classID

In [56]:
combined_val_y = combine_classes([0, 2, 4, 5, 6, 7, 9], val_y)

## 1. Model - AddGaussianNoise

In [3]:
pipe = make_oversampled_feature_pipeline()

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline


In [5]:
pipe.set_params(over_sampler=AugmentingRandomOversampler(),
                estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),
                mean_mfcc__kw_args= {'sr': 16000,
                'n_mfcc': 16,
                'n_fft': 256,
                'hop_length': 128,
                'fmin': 0.0,
                'fmax': 3000.0,
                'n_mels': 98})

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 256,
                                              'n_mfcc': 16, 'sr'...
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.025, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=None, nthread=4, num_parallel_tree=None,
                               objective='multi:softmax', predictor=None,
                               random_state=None, ...))])

In [10]:
pipe.fit(train_fold, train_y)

/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 256,
                                              'n_mfcc': 16, 'sr'...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.025, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=4, nthread=4,
                               num_parallel_tree=1, objective='multi:softmax',
                               predictor='auto', random_state=0, ...))])

In [12]:
pred_y = pipe.predict(val_fold)

/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


In [13]:
print(classification_report(val_y, pred_y))

              precision    recall  f1-score   support

           0       0.47      0.23      0.31       100
           1       0.39      0.51      0.44        59
           2       0.37      0.41      0.39       100
           3       0.44      0.74      0.55       100
           4       0.55      0.55      0.55       100
           5       0.41      0.24      0.30       107
           6       0.70      0.55      0.62        38
           7       0.37      0.12      0.19       120
           8       0.87      0.78      0.82       166
           9       0.25      0.50      0.33       100

    accuracy                           0.47       990
   macro avg       0.48      0.46      0.45       990
weighted avg       0.49      0.47      0.46       990



In [45]:
report1 = pd.DataFrame(classification_report(val_y, pred_y, output_dict=True)).transpose()
mean_report1 = {'mean_precision': np.mean(report1['precision']),
                'mean_recall':np.mean(report1['recall']),
                'mean_f1-score': np.mean(report1['f1-score'])
                }
pd.DataFrame(mean_report1, index=[1])

,mean_precision,mean_recall,mean_f1-score
1,0.480564,0.4644,0.452094


### Combined Class Classification Report

Target class: car horns (1), sirens(8), dog barks(3)

Dangerous classes in report has 0.71 precision, 0.86 recall, 0.78 f1-score. 

This model has 0.52 accuracy. 

In [57]:
com_pred_y1 = combine_classes([0, 2, 4, 5, 6, 7, 9], pred_y)

report1_1 = classification_report(combined_val_y, com_pred_y1)
print(report1_1)

              precision    recall  f1-score   support

           0       0.47      0.23      0.31       100
           2       0.37      0.41      0.39       100
           4       0.55      0.55      0.55       100
           5       0.41      0.24      0.30       107
           6       0.70      0.55      0.62        38
           7       0.37      0.12      0.19       120
           8       0.71      0.86      0.78       325
           9       0.25      0.50      0.33       100

    accuracy                           0.52       990
   macro avg       0.48      0.43      0.43       990
weighted avg       0.51      0.52      0.49       990



## 2. Model -AirAbsorption

In [14]:
pipe2 = make_oversampled_feature_pipeline()

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline


In [19]:
pipe2.set_params(over_sampler=AugmentingRandomOversampler(augment_method='AirAbsorption'),
                estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),
                mean_mfcc__kw_args= {'sr': 16000,
                'n_mfcc': 16,
                'n_fft': 256,
                'hop_length': 128,
                'fmin': 0.0,
                'fmax': 3000.0,
                'n_mels': 98})

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='AirAbsorption',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 1...
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.025, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=None, nthread=4, num_parallel_tree=None,
                               objective='multi:softmax', predictor=None,
                               random_state=None, ...))])

In [20]:
pipe2.fit(train_fold, train_y)

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='AirAbsorption',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 1...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.025, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=4, nthread=4,
                               num_parallel_tree=1, objective='multi:softmax',
                               predictor='auto', random_state=0, ...))])

In [21]:
pred_y2 = pipe2.predict(val_fold)

In [22]:
print(classification_report(val_y, pred_y2))

              precision    recall  f1-score   support

           0       0.36      0.16      0.22       100
           1       0.72      0.36      0.48        59
           2       0.42      0.43      0.43       100
           3       0.45      0.80      0.58       100
           4       0.42      0.56      0.48       100
           5       0.33      0.20      0.25       107
           6       0.76      0.58      0.66        38
           7       0.25      0.12      0.16       120
           8       0.92      0.80      0.85       166
           9       0.24      0.51      0.33       100

    accuracy                           0.46       990
   macro avg       0.49      0.45      0.44       990
weighted avg       0.48      0.46      0.45       990



In [46]:
report2= pd.DataFrame(classification_report(val_y, pred_y2, output_dict=True)).transpose()
mean_report2 = {'mean_precision': np.mean(report2['precision']),
                'mean_recall':np.mean(report2['recall']),
                'mean_f1-score': np.mean(report2['f1-score'])
                }
pd.DataFrame(mean_report2, index=[2])

,mean_precision,mean_recall,mean_f1-score
2,0.485627,0.451885,0.444


### Combined Class Classification Report

Target class: car horns (1), sirens(8), dog barks(3)

Dangerous classes in report has 0.78 precision, 0.84 recall, 0.81 f1-score. 

This model has 0.50 accuracy.

In [58]:
com_pred_y2 = combine_classes([0, 2, 4, 5, 6, 7, 9], pred_y2)

report2_1 = classification_report(combined_val_y, com_pred_y2)
print(report2_1)

              precision    recall  f1-score   support

           0       0.36      0.16      0.22       100
           2       0.42      0.43      0.43       100
           4       0.42      0.56      0.48       100
           5       0.33      0.20      0.25       107
           6       0.76      0.58      0.66        38
           7       0.25      0.12      0.16       120
           8       0.78      0.84      0.81       325
           9       0.24      0.51      0.33       100

    accuracy                           0.50       990
   macro avg       0.45      0.42      0.42       990
weighted avg       0.50      0.50      0.48       990



## 3. Model -Time Stretch 

In [27]:
pipe3 = make_oversampled_feature_pipeline()

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline


In [28]:
pipe3.set_params(over_sampler=AugmentingRandomOversampler(augment_method='TimeStretch'),
                estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),
                mean_mfcc__kw_args= {'sr': 16000,
                'n_mfcc': 16,
                'n_fft': 256,
                'hop_length': 128,
                'fmin': 0.0,
                'fmax': 3000.0,
                'n_mels': 98})

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='TimeStretch',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128...
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.025, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=None, nthread=4, num_parallel_tree=None,
                               objective='multi:softmax', predictor=None,
                               random_state=None, ...))])

In [29]:
pipe3.fit(train_fold, train_y)

/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1105
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1696
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1550
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1600
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=960
  return f(*args, **kwargs)
/Users/carlquin/

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='TimeStretch',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.025, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=4, nthread=4,
                               num_parallel_tree=1, objective='multi:softmax',
                               predictor='auto', random_state=0, ...))])

In [30]:
pred_y3 = pipe3.predict(val_fold)

In [31]:
print(classification_report(val_y, pred_y3))

              precision    recall  f1-score   support

           0       0.35      0.17      0.23       100
           1       0.66      0.36      0.46        59
           2       0.41      0.45      0.43       100
           3       0.50      0.78      0.61       100
           4       0.43      0.57      0.49       100
           5       0.27      0.18      0.21       107
           6       0.78      0.55      0.65        38
           7       0.30      0.13      0.18       120
           8       0.93      0.80      0.86       166
           9       0.24      0.53      0.33       100

    accuracy                           0.46       990
   macro avg       0.49      0.45      0.45       990
weighted avg       0.49      0.46      0.45       990



In [47]:
report3= pd.DataFrame(classification_report(val_y, pred_y3, output_dict=True)).transpose()
mean_report3 = {'mean_precision': np.mean(report3['precision']),
                'mean_recall':np.mean(report3['recall']),
                'mean_f1-score': np.mean(report3['f1-score'])
                }
pd.DataFrame(mean_report3, index=[3])

,mean_precision,mean_recall,mean_f1-score
3,0.485169,0.453337,0.44756


### Combined Class Classification Report

Target class: car horns (1), sirens(8), dog barks(3)

Dangerous classes in report has 0.81 precision, 0.82 recall, 0.82 f1-score. 

This model has 0.50 accuracy.

In [59]:
com_pred_y3 = combine_classes([0, 2, 4, 5, 6, 7, 9], pred_y3)

report3_1 = classification_report(combined_val_y, com_pred_y3)
print(report3_1)

              precision    recall  f1-score   support

           0       0.35      0.17      0.23       100
           2       0.41      0.45      0.43       100
           4       0.43      0.57      0.49       100
           5       0.27      0.18      0.21       107
           6       0.78      0.55      0.65        38
           7       0.30      0.13      0.18       120
           8       0.81      0.82      0.82       325
           9       0.24      0.53      0.33       100

    accuracy                           0.50       990
   macro avg       0.45      0.43      0.42       990
weighted avg       0.51      0.50      0.49       990



## 4. Model -PitchShift

In [32]:
pipe4 = make_oversampled_feature_pipeline()

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline


In [33]:
pipe4.set_params(over_sampler=AugmentingRandomOversampler(augment_method='PitchShift'),
                estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),
                mean_mfcc__kw_args= {'sr': 16000,
                'n_mfcc': 16,
                'n_fft': 256,
                'hop_length': 128,
                'fmin': 0.0,
                'fmax': 3000.0,
                'n_mels': 98})

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='PitchShift',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128,...
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.025, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=None, nthread=4, num_parallel_tree=None,
                               objective='multi:softmax', predictor=None,
                               random_state=None, ...))])

In [34]:
pipe4.fit(train_fold, train_y)

/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1550
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1600
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1696
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=960
  return f(*args, **kwargs)
/Users/carlquin/.pyenv/versions/usk8data/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=800
  return f(*args, **kwargs)
/Users/carlquin/.

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='PitchShift',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128,...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.025, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=4, nthread=4,
                               num_parallel_tree=1, objective='multi:softmax',
                               predictor='auto', random_state=0, ...))])

In [35]:
pred_y4 = pipe4.predict(val_fold)

In [36]:
print(classification_report(val_y, pred_y4))

              precision    recall  f1-score   support

           0       0.38      0.18      0.24       100
           1       0.41      0.44      0.43        59
           2       0.46      0.41      0.43       100
           3       0.48      0.78      0.60       100
           4       0.48      0.57      0.52       100
           5       0.33      0.19      0.24       107
           6       0.70      0.55      0.62        38
           7       0.28      0.12      0.17       120
           8       0.94      0.78      0.85       166
           9       0.24      0.54      0.33       100

    accuracy                           0.46       990
   macro avg       0.47      0.46      0.44       990
weighted avg       0.48      0.46      0.45       990



In [48]:
report4= pd.DataFrame(classification_report(val_y, pred_y4, output_dict=True)).transpose()
mean_report4 = {'mean_precision': np.mean(report4['precision']),
                'mean_recall':np.mean(report4['recall']),
                'mean_f1-score': np.mean(report4['f1-score'])
                }
pd.DataFrame(mean_report4, index=[4])

,mean_precision,mean_recall,mean_f1-score
4,0.470563,0.458037,0.445709


### Combined Class Classification Report

Target class: car horns (1), sirens(8), dog barks(3)

Dangerous classes in report has 0.76 precision, 0.85 recall, 0.80 f1-score. 

This model has 0.51 accuracy.

In [60]:
com_pred_y4 = combine_classes([0, 2, 4, 5, 6, 7, 9], pred_y4)

report4_1 = classification_report(combined_val_y, com_pred_y4)
print(report4_1)

              precision    recall  f1-score   support

           0       0.38      0.18      0.24       100
           2       0.46      0.41      0.43       100
           4       0.48      0.57      0.52       100
           5       0.33      0.19      0.24       107
           6       0.70      0.55      0.62        38
           7       0.28      0.12      0.17       120
           8       0.76      0.85      0.80       325
           9       0.24      0.54      0.33       100

    accuracy                           0.51       990
   macro avg       0.45      0.43      0.42       990
weighted avg       0.50      0.51      0.49       990



## 5. Model -Shift

In [37]:
pipe5 = make_oversampled_feature_pipeline()

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline


In [39]:
pipe5.set_params(over_sampler=AugmentingRandomOversampler(augment_method='Shift'),
                estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),
                mean_mfcc__kw_args= {'sr': 16000,
                'n_mfcc': 16,
                'n_fft': 256,
                'hop_length': 128,
                'fmin': 0.0,
                'fmax': 3000.0,
                'n_mels': 98})

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='Shift',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128, 'n_ff...
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.025, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=None, nthread=4, num_parallel_tree=None,
                               objective='multi:softmax', predictor=None,
                               random_state=None, ...))])

In [40]:
pipe5.fit(train_fold, train_y)

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x15335f0d0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(augment_method='Shift',
                                             sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x10cf3f790>,
                                     kw_args={'fmax': 3000.0, 'fmin': 0.0,
                                              'hop_length': 128, 'n_ff...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.025, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=4, nthread=4,
                               num_parallel_tree=1, objective='multi:softmax',
                               predictor='auto', random_state=0, ...))])

In [41]:
pred_y5 = pipe5.predict(val_fold)

In [42]:
print(classification_report(val_y, pred_y5))

              precision    recall  f1-score   support

           0       0.35      0.16      0.22       100
           1       0.66      0.32      0.43        59
           2       0.41      0.45      0.43       100
           3       0.46      0.77      0.57       100
           4       0.45      0.56      0.50       100
           5       0.30      0.19      0.23       107
           6       0.75      0.55      0.64        38
           7       0.27      0.12      0.17       120
           8       0.92      0.82      0.87       166
           9       0.23      0.49      0.31       100

    accuracy                           0.46       990
   macro avg       0.48      0.44      0.44       990
weighted avg       0.48      0.46      0.45       990



In [49]:
report5= pd.DataFrame(classification_report(val_y, pred_y5, output_dict=True)).transpose()
mean_report5 = {'mean_precision': np.mean(report5['precision']),
                'mean_recall':np.mean(report5['recall']),
                'mean_f1-score': np.mean(report5['f1-score'])
                }
pd.DataFrame(mean_report5, index=[5])

,mean_precision,mean_recall,mean_f1-score
5,0.476665,0.445894,0.439103


### Combined Class Classification Report

Target class: car horns (1), sirens(8), dog barks(3)

Dangerous classes in report has 0.78 precision, 0.83 recall, 0.81 f1-score. 

This model has 0.50 accuracy.

In [61]:
com_pred_y5 = combine_classes([0, 2, 4, 5, 6, 7, 9], pred_y5)

report5_1 = classification_report(combined_val_y, com_pred_y5)
print(report5_1)

              precision    recall  f1-score   support

           0       0.35      0.16      0.22       100
           2       0.41      0.45      0.43       100
           4       0.45      0.56      0.50       100
           5       0.30      0.19      0.23       107
           6       0.75      0.55      0.64        38
           7       0.27      0.12      0.17       120
           8       0.78      0.83      0.81       325
           9       0.23      0.49      0.31       100

    accuracy                           0.50       990
   macro avg       0.44      0.42      0.41       990
weighted avg       0.50      0.50      0.48       990



## Conclusion

AirAbsorption has best mean precision and AddGaussionNoise has best mean recall and mean f1-score.

When looking at combined classes (target class: car horns, sirens, dog barks), TimeStretch has best precision(0.81) and f1-score(0.82), AddGaussianNoise has best recall(0.86) on the combined target class.